In [ ]:
# initiate the k-means

In [1]:
import csv
import numpy as np
import pandas as pd
from math import factorial

import datetime
from datetime import date
import datetime
import time

import scipy
import os, os.path
from os import listdir
from os.path import isfile, join

import re
# from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sb

import sys


# search path for modules
# look @ https://stackoverflow.com/questions/67631/how-to-import-a-module-given-the-full-path
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
import NASA_plot_core as npc

In [2]:
size = 20
title_FontSize = 10
legend_FontSize = 14
tick_FontSize = 18
label_FontSize = 14

params = {'legend.fontsize': 17,
          'figure.figsize': (6, 4),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size * 0.75,
          'ytick.labelsize': size * 0.75,
          'axes.titlepad': 10}

#
#  Once set, you cannot change them, unless restart the notebook
#
plt.rc('font', family = 'Palatino')
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['xtick.labelbottom'] = True
plt.rcParams['ytick.labelleft'] = True
plt.rcParams.update(params)

In [8]:
data_dir = "/Users/hn/Documents/01_research_data/NASA/data_deBug/05_SG_TS/"

In [9]:
df = pd.read_csv(data_dir + "SG_AdamBenton2016_EVI_JFD.csv")
df['human_system_start_time'] = pd.to_datetime(df['human_system_start_time'])

In [10]:
df.head(2)

,ID,human_system_start_time,EVI
0,10000_WSDA_SF_2016,2015-01-07,0.139519
1,10000_WSDA_SF_2016,2015-01-17,0.145918


# Harmonize the x-values!!!

In the regularization step we bin the date range by looking at the range of images. Therefore, for a given field
we might have a 10-day bin that goes from Jan 1 to Jan 10 and for another field it goes from Jan 8 to Jan 17. Therefore, let us just to assume the dates are the same as opposed to changing the code to fit the damn clustering step! Anyway, we are picking maximum value of a VI in a 10-day window. The date/x-values are not exact anyway!

In [11]:
fields_IDs = df.ID.unique()

In [26]:
field_0 = df[df.ID == fields_IDs[0]].copy()
field_0 = field_0[field_0['human_system_start_time'].dt.year == 2016].copy()
print (field_0.shape)
field_0.head(2)

(37, 3)


,ID,human_system_start_time,EVI
36,10000_WSDA_SF_2016,2016-01-02,0.140782
37,10000_WSDA_SF_2016,2016-01-12,0.129974


In [56]:
field_1 = df[df.ID == fields_IDs[20]].copy()
field_1 = field_1[field_1['human_system_start_time'].dt.year == 2016].copy()
print (field_1.shape)

(36, 3)


In [66]:
field_1 = df[df.ID == fields_IDs[21]]
field_1 = field_1[field_1['human_system_start_time'].dt.year == 2016].copy()
print (field_1.shape)

(37, 3)


In [67]:
print (df.shape)
df.drop(field_1.index[35:], inplace=True)
print (df.shape)

(2148882, 3)
(2148880, 3)


In [74]:
print ("it took {:.0f} seconds to run this code.".format(0.1234))

it took 0 seconds to run this code.


In [75]:
a = 1201026676960 / 1000
time.strftime('%Y-%m-%d', time.localtime(a))

'2008-01-22'

In [109]:
A = pd.read_csv("/Users/hn/Documents/01_research_data/NASA/" + \
                "VI_TS/07_2crop_acr/doubleAcr_perCounty_perCrop_EVI_JFD.csv")
A.head(2)

,county,CropTyp,year,season_count,threshold,ExctAcr
0,Adams,alfalfa hay,2016,1,0.3,12958.262339
1,Adams,alfalfa hay,2016,2,0.3,1466.917252


In [111]:
A.head(2)

,county,CropTyp,year,season_count,threshold,ExctAcr
0,Adams,alfalfa hay,2016,1,0.3,12958.0
1,Adams,alfalfa hay,2016,2,0.3,1467.0


In [112]:
A.shape

(1773, 6)

In [114]:
A.CropTyp.unique

<bound method Series.unique of 0             alfalfa hay
1             alfalfa hay
2            alfalfa seed
3            alfalfa seed
4       alfalfa/grass hay
              ...        
1768               walnut
1769                wheat
1770                wheat
1771         wheat fallow
1772        wildlife feed
Name: CropTyp, Length: 1773, dtype: object>

In [115]:
B = A[A.CropTyp != "alfalfa hay"]

In [117]:
B.CropTyp.unique

<bound method Series.unique of 2            alfalfa seed
3            alfalfa seed
4       alfalfa/grass hay
5                   apple
6               asparagus
              ...        
1768               walnut
1769                wheat
1770                wheat
1771         wheat fallow
1772        wildlife feed
Name: CropTyp, Length: 1737, dtype: object>